# MLflow Judges Feature Demo

This notebook demonstrates the complete MLflow Judges workflow including:
1. Setting up an MLflow tracking server
2. Creating a simple chat bot with tracing
3. Providing human feedback on traces
4. Creating and registering judges
5. Running evaluations
6. Aligning judges based on feedback
7. Re-evaluating with aligned judges

## Prerequisites

Before running this notebook, start the MLflow tracking server in a terminal:

```bash
# Terminal 1: Start MLflow tracking server
rm -f demo_mlflow.db  # Start fresh
uv run mlflow server \
  --backend-store-uri sqlite:///demo_mlflow.db \
  --default-artifact-root mlruns \
  --host 127.0.0.1 \
  --port 5000

# Terminal 2 (optional): Start React dev server for latest UI
cd mlflow/server/js
yarn start  # Will run on http://localhost:3000
```

The notebook will connect to the tracking server on port 5000.

## Setup and Configuration

In [1]:
import os
import time

import mlflow
from mlflow import MlflowClient
from mlflow.genai.judges import make_judge
import anthropic

/Users/benjamin.wilson/repos/mlflow-fork/mlflow/mlflow/models/utils.py:51: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.3)
  from scipy.sparse import csc_matrix, csr_matrix


In [2]:
# Connect to existing MLflow tracking server
TRACKING_URI = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(TRACKING_URI)
client = MlflowClient(tracking_uri=TRACKING_URI)

# Verify connection
try:
    experiments = client.search_experiments()
    print(f"✅ Connected to MLflow tracking server at {TRACKING_URI}")
    print(f"Found {len(experiments)} existing experiments")
except Exception as e:
    print(f"❌ Could not connect to MLflow server at {TRACKING_URI}")
    print(f"Error: {e}")
    print("\nMake sure the server is running with:")
    print("  uv run mlflow server --backend-store-uri sqlite:///demo_mlflow.db --port 5000")

✅ Connected to MLflow tracking server at http://127.0.0.1:5000
Found 5 existing experiments


## Create Experiment

In [3]:
# Create or get experiment for the demo
experiment_name = "judges-demo-5"

# Ensure we're using the right tracking URI
mlflow.set_tracking_uri(TRACKING_URI)

# Try to get existing experiment first
existing_experiment = mlflow.get_experiment_by_name(experiment_name)
if existing_experiment:
    experiment = existing_experiment.experiment_id
    print(f"Using existing experiment: {experiment_name} (ID: {experiment})")
else:
    experiment = mlflow.create_experiment(
        experiment_name,
        tags={"demo": "judges", "purpose": "demonstration"}
    )
    print(f"Created new experiment: {experiment_name} (ID: {experiment})")

# Set the experiment as active
mlflow.set_experiment(experiment_name)
print(f"\nActive experiment: {experiment_name} (ID: {experiment})")

Created new experiment: judges-demo-5 (ID: 6)

Active experiment: judges-demo-5 (ID: 6)


## Simple Chat Bot Implementation with Anthropic

In [4]:
# Initialize Anthropic client
anthropic_client = anthropic.Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY")
)

# Enable MLflow autologging for Anthropic
mlflow.anthropic.autolog(
    log_traces=True,
    disable=False
)

print("Anthropic client initialized and MLflow autologging enabled")

Anthropic client initialized and MLflow autologging enabled


In [5]:
# Simple chat function - autolog will handle tracing automatically
def chat_with_assistant(user_message: str, system_prompt: str = None) -> str:
    """Simple chat function that uses Anthropic's Claude"""
    
    messages = [{"role": "user", "content": user_message}]
    
    # Create chat completion - MLflow autolog will trace this automatically
    response = anthropic_client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=500,
        temperature=0.7,
        system=system_prompt or "You are a helpful assistant.",
        messages=messages
    )
    
    # Extract response text
    response_text = response.content[0].text
    
    return response_text

print("Chat function defined (tracing handled by autolog)")

Chat function defined (tracing handled by autolog)


## Generate Chat Interactions and Traces

In [6]:
# Test conversations to generate traces
test_conversations = [
    {
        "user": "What is machine learning?",
        "system": "You are an AI educator. Provide clear, concise explanations."
    },
    {
        "user": "How do I train a neural network?",
        "system": "You are a technical mentor. Give practical advice."
    },
    {
        "user": "What are the benefits of using MLflow?",
        "system": "You are an MLOps expert. Explain tools and best practices."
    },
    {
        "user": "Can you explain gradient descent?",
        "system": "You are a math tutor. Explain concepts simply."
    },
    {
        "user": "What is the difference between supervised and unsupervised learning?",
        "system": "You are an AI educator. Compare and contrast clearly."
    },
    {
        "user": "How do I choose the right algorithm for my problem?",
        "system": "You are a data science consultant. Provide practical guidance."
    },
    {
        "user": "What is overfitting and how can I prevent it?",
        "system": "You are a machine learning expert. Explain clearly with examples."
    },
    {
        "user": "How do I evaluate model performance?",
        "system": "You are a model evaluation specialist. Give comprehensive advice."
    },
    {
        "user": "What is feature engineering?",
        "system": "You are a data preprocessing expert. Explain with practical tips."
    },
    {
        "user": "How do I handle missing data in my dataset?",
        "system": "You are a data cleaning specialist. Provide actionable solutions."
    }
]

print("Generating chat traces...\n")
responses = []

# Ensure we're in the right experiment
mlflow.set_experiment(experiment_name)

with mlflow.start_run():
    for i, conv in enumerate(test_conversations):
        print(f"Chat {i+1}: {conv['user'][:50]}...")
        response = chat_with_assistant(
            user_message=conv['user'],
            system_prompt=conv['system']
        )
        responses.append({
            "user": conv['user'],
            "system": conv['system'],
            "response": response
        })
        print(f"Response: {response[:100]}...\n")
        time.sleep(1)  # Small delay between requests

print(f"\nGenerated {len(responses)} chat traces")

Generating chat traces...

Chat 1: What is machine learning?...
Response: Machine learning is a field of artificial intelligence that enables computers and systems to learn a...

Chat 2: How do I train a neural network?...
Response: Training a neural network can be a complex process, but here are some practical steps you can take:
...

Chat 3: What are the benefits of using MLflow?...
Response: MLflow is a popular open-source platform for managing the end-to-end machine learning lifecycle. Her...

Chat 4: Can you explain gradient descent?...
Response: Certainly! Gradient descent is a fundamental optimization algorithm used in machine learning and oth...

Chat 5: What is the difference between supervised and unsu...
Response: The main difference between supervised and unsupervised learning lies in the way the algorithms are ...

Chat 6: How do I choose the right algorithm for my problem...
Response: Choosing the right algorithm for your problem is a crucial step in the data science proc

[Trace(trace_id=tr-9c07d31ddb3c6f298e23e52e9dd9a1c7), Trace(trace_id=tr-08a85a07192aaef31765bfaf51f46be0), Trace(trace_id=tr-fda4882e30a9296a5195712b486d9b15), Trace(trace_id=tr-be002759342df899bfeceafd9ff35384), Trace(trace_id=tr-4af75fd704dec9956c43678297ad89a2), Trace(trace_id=tr-0b4dac4b8e533a3a51e5ca19d0f7d157), Trace(trace_id=tr-485740977e1e5fe2ac55c4aad5d65817), Trace(trace_id=tr-c484ba2dd6d07b61687a0ef266d589e1), Trace(trace_id=tr-ca442a03b7eee40424d0b17b19d98649), Trace(trace_id=tr-52fc3f9c26c0e6a43ca0fff5f1b1ec9d)]

## Fetch Traces and Add Human Feedback

In [7]:
# Fetch traces from the experiment
traces = client.search_traces(
    experiment_ids=[experiment],
    max_results=10
)

print(f"Found {len(traces)} traces")
print("\nTrace IDs:")
for trace in traces:
    print(f"  - {trace.info.request_id}")

Found 10 traces

Trace IDs:
  - tr-52fc3f9c26c0e6a43ca0fff5f1b1ec9d
  - tr-ca442a03b7eee40424d0b17b19d98649
  - tr-c484ba2dd6d07b61687a0ef266d589e1
  - tr-485740977e1e5fe2ac55c4aad5d65817
  - tr-0b4dac4b8e533a3a51e5ca19d0f7d157
  - tr-4af75fd704dec9956c43678297ad89a2
  - tr-be002759342df899bfeceafd9ff35384
  - tr-fda4882e30a9296a5195712b486d9b15
  - tr-08a85a07192aaef31765bfaf51f46be0
  - tr-9c07d31ddb3c6f298e23e52e9dd9a1c7


In [8]:
# Add expectations (ground truth) to traces
from mlflow.entities import AssessmentSource, AssessmentSourceType

# Define expectations for each conversation
expectations = [
    {
        "expected_response": "Machine learning is a subset of artificial intelligence that enables computers to learn from data without explicit programming.",
        "key_concepts": ["subset of AI", "learning from data", "pattern recognition"],
        "quality_criteria": {"clarity": "high", "accuracy": "high", "completeness": "medium"}
    },
    {
        "expected_response": "Training a neural network involves forward propagation, loss calculation, backpropagation, and weight updates through optimization.",
        "key_concepts": ["forward propagation", "backpropagation", "optimization", "loss function"],
        "quality_criteria": {"clarity": "high", "accuracy": "high", "technical_depth": "medium"}
    },
    {
        "expected_response": "MLflow provides experiment tracking, model versioning, deployment capabilities, and centralized model registry.",
        "key_concepts": ["experiment tracking", "model registry", "deployment", "versioning"],
        "quality_criteria": {"clarity": "high", "accuracy": "high", "practical_focus": "high"}
    },
    {
        "expected_response": "Gradient descent is an optimization algorithm that iteratively adjusts parameters to minimize a loss function.",
        "key_concepts": ["optimization", "iterative process", "loss minimization", "parameter updates"],
        "quality_criteria": {"clarity": "high", "simplicity": "high", "mathematical_accuracy": "high"}
    },
    {
        "expected_response": "Supervised learning uses labeled data while unsupervised learning finds patterns in unlabeled data.",
        "key_concepts": ["labeled vs unlabeled", "prediction vs pattern discovery", "training differences"],
        "quality_criteria": {"clarity": "high", "comparison": "clear", "completeness": "high"}
    },
    {
        "expected_response": "Algorithm choice depends on problem type, data size, interpretability needs, and performance requirements.",
        "key_concepts": ["problem type", "data characteristics", "business requirements", "performance trade-offs"],
        "quality_criteria": {"clarity": "high", "practicality": "high", "comprehensiveness": "medium"}
    },
    {
        "expected_response": "Overfitting occurs when models memorize training data instead of learning patterns, prevented by validation and regularization.",
        "key_concepts": ["memorization vs generalization", "validation techniques", "regularization methods"],
        "quality_criteria": {"clarity": "high", "accuracy": "high", "practical_advice": "high"}
    },
    {
        "expected_response": "Model evaluation uses metrics like accuracy, precision, recall, F1-score, and cross-validation techniques.",
        "key_concepts": ["evaluation metrics", "cross-validation", "train/test splits", "metric selection"],
        "quality_criteria": {"clarity": "high", "completeness": "high", "technical_accuracy": "high"}
    },
    {
        "expected_response": "Feature engineering involves creating, selecting, and transforming variables to improve model performance.",
        "key_concepts": ["feature creation", "feature selection", "transformations", "domain knowledge"],
        "quality_criteria": {"clarity": "high", "practicality": "high", "comprehensiveness": "medium"}
    },
    {
        "expected_response": "Handle missing data through deletion, imputation, or flagging strategies based on data patterns and business context.",
        "key_concepts": ["missing data patterns", "imputation methods", "deletion strategies", "business impact"],
        "quality_criteria": {"clarity": "high", "practicality": "high", "completeness": "high"}
    }
]

print("Adding expectations to traces...\n")

# Fetch all traces to get the latest count
all_traces = client.search_traces(
    experiment_ids=[experiment],
    max_results=20  # Get enough to cover all new traces
)

print(f"Found {len(all_traces)} total traces")

for i, trace in enumerate(all_traces[:10]):  # Process up to 10 traces
    if i < len(expectations):
        expectation = expectations[i]
        
        # Log expectation for the trace - FIXED: use keyword arguments, not Assessment object
        mlflow.log_expectation(
            trace_id=trace.info.request_id,
            name="expected_response",
            value={
                "ideal_answer": expectation["expected_response"],
                "key_concepts": expectation["key_concepts"],
                "quality_criteria": expectation["quality_criteria"]
            },
            source=AssessmentSource(
                source_type=AssessmentSourceType.HUMAN,
                source_id="domain_expert"
            ),
            metadata={
                "question_index": i,
                "difficulty": "medium" if i in [1, 3, 6, 7] else "easy"
            }
        )
        
        print(f"Added expectation to trace {i+1}: {len(expectation['key_concepts'])} key concepts defined")

print(f"\nExpectations added to {min(len(all_traces), len(expectations))} traces using mlflow.log_expectation()")

Adding expectations to traces...

Found 10 total traces
Added expectation to trace 1: 3 key concepts defined
Added expectation to trace 2: 4 key concepts defined
Added expectation to trace 3: 4 key concepts defined
Added expectation to trace 4: 4 key concepts defined
Added expectation to trace 5: 3 key concepts defined
Added expectation to trace 6: 4 key concepts defined
Added expectation to trace 7: 3 key concepts defined
Added expectation to trace 8: 4 key concepts defined
Added expectation to trace 9: 4 key concepts defined
Added expectation to trace 10: 4 key concepts defined

Expectations added to 10 traces using mlflow.log_expectation()


In [9]:
# Add human feedback to traces using the proper feedback API
from mlflow.entities import AssessmentSource, AssessmentSourceType

feedback_data = [
    {"score": 5, "helpfulness": 5, "accuracy": 5, "clarity": 5, "comment": "Excellent explanation, very clear"},
    {"score": 4, "helpfulness": 4, "accuracy": 5, "clarity": 4, "comment": "Good technical detail, could be simpler"},
    {"score": 5, "helpfulness": 5, "accuracy": 5, "clarity": 5, "comment": "Perfect MLOps explanation"},
    {"score": 3, "helpfulness": 3, "accuracy": 4, "clarity": 3, "comment": "Needs more examples"},
    {"score": 5, "helpfulness": 5, "accuracy": 5, "clarity": 4, "comment": "Great comparison, very useful"},
    {"score": 4, "helpfulness": 4, "accuracy": 4, "clarity": 4, "comment": "Solid practical guidance"},
    {"score": 3, "helpfulness": 3, "accuracy": 4, "clarity": 3, "comment": "Good concept but needs more examples"},
    {"score": 5, "helpfulness": 5, "accuracy": 5, "clarity": 5, "comment": "Comprehensive evaluation guide"},
    {"score": 4, "helpfulness": 4, "accuracy": 4, "clarity": 4, "comment": "Practical feature engineering tips"},
    {"score": 4, "helpfulness": 4, "accuracy": 4, "clarity": 4, "comment": "Good strategies for missing data"}
]

print("Adding human feedback to traces...\n")

# Fetch all traces for feedback
all_traces = client.search_traces(
    experiment_ids=[experiment],
    max_results=20
)

print(f"Found {len(all_traces)} total traces for feedback")

for i, trace in enumerate(all_traces[:10]):  # Process up to 10 traces
    if i < len(feedback_data):
        feedback = feedback_data[i]
        
        # Log human feedback with overall score and detailed breakdown
        mlflow.log_feedback(
            trace_id=trace.info.request_id,
            name="human_evaluation",
            value=feedback["score"],  # Single numeric score for UI display
            source=AssessmentSource(
                source_type=AssessmentSourceType.HUMAN,
                source_id="demo_evaluator"
            ),
            rationale=feedback["comment"],  # Human's reasoning for the score
            metadata={
                "evaluation_round": 1,
                "helpfulness": feedback["helpfulness"],
                "accuracy": feedback["accuracy"],
                "clarity": feedback["clarity"]
            }
        )
        
        print(f"Added feedback to trace {i+1}: Score={feedback['score']}, {feedback['comment']}")

print(f"\nHuman feedback added to {min(len(all_traces), len(feedback_data))} traces using mlflow.log_feedback()")

Adding human feedback to traces...

Found 10 total traces for feedback
Added feedback to trace 1: Score=5, Excellent explanation, very clear
Added feedback to trace 2: Score=4, Good technical detail, could be simpler
Added feedback to trace 3: Score=5, Perfect MLOps explanation
Added feedback to trace 4: Score=3, Needs more examples
Added feedback to trace 5: Score=5, Great comparison, very useful
Added feedback to trace 6: Score=4, Solid practical guidance
Added feedback to trace 7: Score=3, Good concept but needs more examples
Added feedback to trace 8: Score=5, Comprehensive evaluation guide
Added feedback to trace 9: Score=4, Practical feature engineering tips
Added feedback to trace 10: Score=4, Good strategies for missing data

Human feedback added to 10 traces using mlflow.log_feedback()


## Create Judges Using make_judge API

In [10]:
# Create Judge 1: Simple helpfulness judge
print("Creating Judge 1: Simple Helpfulness Judge...")

judge1 = make_judge(
    name="helpfulness_judge",
    instructions="""
    Rate how helpful the response is for the question asked.
    
    Question: {{ inputs }}
    Response: {{ outputs }}
    
    Score 1-5:
    - 5: Very helpful, directly answers the question
    - 4: Helpful, mostly answers the question  
    - 3: Somewhat helpful, partially answers
    - 2: Not very helpful, misses key points
    - 1: Not helpful, doesn't answer the question
    """,
    model="openai:/gpt-4o-mini"
)

print("Judge 1 created successfully")

Creating Judge 1: Simple Helpfulness Judge...
Judge 1 created successfully


In [11]:
# Create Judge 2: Simple accuracy judge using expectations
print("Creating Judge 2: Simple Accuracy Judge...")

judge2 = make_judge(
    name="accuracy_judge",
    instructions="""
    Compare the response to the expected answer.
    
    Response: {{ outputs }}
    Expected: {{ expectations }}
    
    Score 1-5 based on accuracy:
    - 5: Matches expected answer closely
    - 4: Mostly accurate
    - 3: Partially accurate
    - 2: Some inaccuracies  
    - 1: Inaccurate
    """,
    model="openai:/gpt-4o-mini"
)

print("Judge 2 created successfully")

Creating Judge 2: Simple Accuracy Judge...
Judge 2 created successfully


## Run Evaluation with Both Judges

In [12]:
# Evaluate traces with Judge 1 (Helpfulness)
print("Step 1: Evaluating traces with Helpfulness Judge\n")
print("=" * 60)

# Get traces for evaluation
traces_list = client.search_traces(
    experiment_ids=[experiment],
    max_results=10  # Get all 10 traces
)

print(f"Found {len(traces_list)} traces to evaluate\n")

# Evaluate with Judge 1
for i, trace in enumerate(traces_list):
    print(f"Trace {i+1}/{len(traces_list)}: ", end="")
    
    try:
        # Evaluate with helpfulness judge
        result = judge1(trace=trace)
        
        # Log the assessment
        mlflow.log_assessment(trace_id=trace.info.request_id, assessment=result)
        print(f"✓ Score={result.value}")
        
    except Exception as e:
        print(f"✗ Error: {str(e)[:50]}")
    
    time.sleep(0.5)  # Small delay to avoid rate limits

print("\n✅ Helpfulness evaluation complete!")

Step 1: Evaluating traces with Helpfulness Judge

Found 10 traces to evaluate

Trace 1/10: ✓ Score=5
Trace 2/10: ✓ Score=5
Trace 3/10: ✓ Score=5
Trace 4/10: ✓ Score=5
Trace 5/10: ✓ Score=5
Trace 6/10: ✓ Score=5
Trace 7/10: ✓ Score=5
Trace 8/10: ✓ Score=5
Trace 9/10: ✓ Score=5
Trace 10/10: ✓ Score=5

✅ Helpfulness evaluation complete!


In [13]:
# Evaluate traces with Judge 2 (Accuracy)
print("\nStep 2: Evaluating traces with Accuracy Judge\n")
print("=" * 60)

# Evaluate with Judge 2
for i, trace in enumerate(traces_list):
    print(f"Trace {i+1}/{len(traces_list)}: ", end="")
    
    try:
        # Evaluate with accuracy judge (compares to expectations)
        result = judge2(trace=trace)
        
        # Log the assessment
        mlflow.log_assessment(trace_id=trace.info.request_id, assessment=result)
        print(f"✓ Score={result.value}")
        
    except Exception as e:
        print(f"✗ Error: {str(e)[:50]}")
    
    time.sleep(0.5)  # Small delay to avoid rate limits

print("\n✅ Accuracy evaluation complete!")


Step 2: Evaluating traces with Accuracy Judge

Trace 1/10: ✓ Score=1
Trace 2/10: ✓ Score=1
Trace 3/10: ✓ Score=1
Trace 4/10: ✓ Score=1
Trace 5/10: ✓ Score=1
Trace 6/10: ✓ Score=1
Trace 7/10: ✓ Score=1
Trace 8/10: ✓ Score=1
Trace 9/10: ✓ Score=1
Trace 10/10: ✓ Score=1

✅ Accuracy evaluation complete!


## Viewing Evaluation Results

In [14]:
# Add human feedback on judge assessments for alignment
print("Adding human feedback on judge assessments for alignment\n")
print("=" * 60)

# Simulate human reviewing the judge results and providing feedback
human_feedback_on_helpfulness_judge = [
    {"helpfulness_judge": 4, "comment": "Judge scored too high, response was good but not excellent"},
    {"helpfulness_judge": 4, "comment": "Judge scored appropriately, good technical content"},
    {"helpfulness_judge": 5, "comment": "Judge scored correctly, excellent MLflow explanation"}, 
    {"helpfulness_judge": 3, "comment": "Judge scored too high, response lacked examples"},
    {"helpfulness_judge": 4, "comment": "Judge scored well, good comparison but not perfect"},
    {"helpfulness_judge": 4, "comment": "Judge scored correctly, practical advice given"},
    {"helpfulness_judge": 2, "comment": "Judge scored too high, explanation was too basic"},
    {"helpfulness_judge": 5, "comment": "Judge scored correctly, very comprehensive guide"},
    {"helpfulness_judge": 4, "comment": "Judge scored well, good practical tips"},
    {"helpfulness_judge": 3, "comment": "Judge scored too high, could use more specific examples"}
]

print("Adding human feedback on helpfulness judge assessments...")

# Get fresh traces list for consistent ordering
fresh_traces_list = client.search_traces(
    experiment_ids=[experiment],
    max_results=10
)

for i, trace in enumerate(fresh_traces_list[:10]):
    if i < len(human_feedback_on_helpfulness_judge):
        feedback = human_feedback_on_helpfulness_judge[i]
        
        # Log human feedback specifically on the judge's assessment
        mlflow.log_feedback(
            trace_id=trace.info.request_id,
            name="helpfulness_judge",  # Same name as the judge for alignment
            value=feedback["helpfulness_judge"],
            source=AssessmentSource(
                source_type=AssessmentSourceType.HUMAN,
                source_id="human_evaluator"
            ),
            rationale=feedback["comment"],
            metadata={"alignment_training": True, "judge_review": True}
        )
        
        print(f"Trace {i+1}: Human scored helpfulness judge {feedback['helpfulness_judge']} - {feedback['comment']}")

print("\n✅ Human feedback on helpfulness judge assessments added!")

Adding human feedback on judge assessments for alignment

Adding human feedback on helpfulness judge assessments...
Trace 1: Human scored helpfulness judge 4 - Judge scored too high, response was good but not excellent
Trace 2: Human scored helpfulness judge 4 - Judge scored appropriately, good technical content
Trace 3: Human scored helpfulness judge 5 - Judge scored correctly, excellent MLflow explanation
Trace 4: Human scored helpfulness judge 3 - Judge scored too high, response lacked examples
Trace 5: Human scored helpfulness judge 4 - Judge scored well, good comparison but not perfect
Trace 6: Human scored helpfulness judge 4 - Judge scored correctly, practical advice given
Trace 7: Human scored helpfulness judge 2 - Judge scored too high, explanation was too basic
Trace 8: Human scored helpfulness judge 5 - Judge scored correctly, very comprehensive guide
Trace 9: Human scored helpfulness judge 4 - Judge scored well, good practical tips
Trace 10: Human scored helpfulness judge 3

In [15]:
# Add human feedback on accuracy judge assessments
print("\nAdding human feedback on accuracy judge assessments for alignment\n")
print("=" * 60)

# The accuracy judge is being too harsh - it's giving mostly 1s because it expects exact matches
# Human feedback should indicate the judge is scoring too low when responses are actually accurate
human_feedback_on_accuracy_judge = [
    {"accuracy_judge": 4, "comment": "Judge scored too low, response covers key ML concepts accurately"},
    {"accuracy_judge": 3, "comment": "Judge scored too low, technical details are mostly correct"},
    {"accuracy_judge": 4, "comment": "Judge scored correctly, good MLflow coverage"},
    {"accuracy_judge": 3, "comment": "Judge scored too low, covers optimization concepts well"},
    {"accuracy_judge": 4, "comment": "Judge scored too low, comparison is technically accurate"},
    {"accuracy_judge": 3, "comment": "Judge scored too low, practical guidance aligns with expectations"},
    {"accuracy_judge": 3, "comment": "Judge scored too low, covers key overfitting concepts"},
    {"accuracy_judge": 4, "comment": "Judge scored too low, evaluation metrics are comprehensively covered"},
    {"accuracy_judge": 4, "comment": "Judge scored too low, feature engineering concepts are accurate"},
    {"accuracy_judge": 3, "comment": "Judge scored too low, missing data strategies are sound"}
]

print("Adding human feedback on accuracy judge assessments...")

for i, trace in enumerate(fresh_traces_list[:10]):
    if i < len(human_feedback_on_accuracy_judge):
        feedback = human_feedback_on_accuracy_judge[i]
        
        # Log human feedback specifically on the accuracy judge's assessment
        mlflow.log_feedback(
            trace_id=trace.info.request_id,
            name="accuracy_judge",  # Same name as the accuracy judge for alignment
            value=feedback["accuracy_judge"],
            source=AssessmentSource(
                source_type=AssessmentSourceType.HUMAN,
                source_id="human_evaluator"
            ),
            rationale=feedback["comment"],
            metadata={"alignment_training": True, "judge_review": True, "judge_type": "accuracy"}
        )
        
        print(f"Trace {i+1}: Human scored accuracy judge {feedback['accuracy_judge']} - {feedback['comment']}")

print("\n✅ Human feedback on accuracy judge assessments added!")
print("\n📋 Summary:")
print("- Helpfulness judge: Tends to score too high (needs to be more critical)")
print("- Accuracy judge: Tends to score too low (needs to be less strict about exact matches)")
print("- Both judges will benefit from alignment with human feedback")


Adding human feedback on accuracy judge assessments for alignment

Adding human feedback on accuracy judge assessments...
Trace 1: Human scored accuracy judge 4 - Judge scored too low, response covers key ML concepts accurately
Trace 2: Human scored accuracy judge 3 - Judge scored too low, technical details are mostly correct
Trace 3: Human scored accuracy judge 4 - Judge scored correctly, good MLflow coverage
Trace 4: Human scored accuracy judge 3 - Judge scored too low, covers optimization concepts well
Trace 5: Human scored accuracy judge 4 - Judge scored too low, comparison is technically accurate
Trace 6: Human scored accuracy judge 3 - Judge scored too low, practical guidance aligns with expectations
Trace 7: Human scored accuracy judge 3 - Judge scored too low, covers key overfitting concepts
Trace 8: Human scored accuracy judge 4 - Judge scored too low, evaluation metrics are comprehensively covered
Trace 9: Human scored accuracy judge 4 - Judge scored too low, feature enginee

## Judge Alignment

Now let's demonstrate how to align judges based on human feedback

In [16]:
# Align the accuracy judge using SIMBA optimizer
from mlflow.genai.judges.optimizers import SIMBAAlignmentOptimizer

print("Aligning Accuracy Judge with human feedback using SIMBA optimizer\n")
print("=" * 60)
print("\n📋 Context:")
print("- Helpfulness judge: Already gives reasonable scores (4-5 range)")
print("- Accuracy judge: Gives all 1s - needs alignment to fix overly strict scoring")
print("\nWe'll only align the accuracy judge to save time (alignment takes ~10 minutes)\n")

# Create an alignment optimizer
optimizer = SIMBAAlignmentOptimizer()

# Refresh traces to get updated assessments
print("Refreshing traces to get latest assessments...")
fresh_traces = client.search_traces(
    experiment_ids=[experiment],
    max_results=10
)

# Check alignment readiness for accuracy judge
alignment_ready_traces = []
for trace in fresh_traces:
    full_trace = mlflow.get_trace(trace.info.request_id)
    if hasattr(full_trace.info, 'assessments') and full_trace.info.assessments:
        has_judge_assessment = any(
            assessment.name == "accuracy_judge" and str(assessment.source.source_type) == "LLM_JUDGE"
            for assessment in full_trace.info.assessments
        )
        has_human_assessment = any(
            assessment.name == "accuracy_judge" and str(assessment.source.source_type) == "HUMAN"
            for assessment in full_trace.info.assessments
        )
        
        if has_judge_assessment and has_human_assessment:
            alignment_ready_traces.append(full_trace)

print(f"Found {len(alignment_ready_traces)} traces ready for accuracy judge alignment")

if len(alignment_ready_traces) >= 10:
    print(f"\n📊 ALIGNING ACCURACY JUDGE")
    print("=" * 40)
    
    print("\nBEFORE ALIGNMENT:")
    print("-" * 20)
    print("Original Instructions:")
    print(judge2.instructions)
    
    print("\n⏳ Starting alignment (this will take ~10 minutes)...")
    
    # Align the accuracy judge
    aligned_accuracy_judge = judge2.align(optimizer, alignment_ready_traces)
    
    print("\n✅ Alignment complete!")
    
    print("\nAFTER ALIGNMENT:")
    print("-" * 20)
    print("Aligned Instructions:")
    print(aligned_accuracy_judge.instructions)
    
    # Test the aligned judge on one trace
    print("\n📈 Testing Aligned Judge:")
    print("-" * 20)
    test_trace = alignment_ready_traces[0]
    
    original_result = judge2(trace=test_trace)
    aligned_result = aligned_accuracy_judge(trace=test_trace)
    
    print(f"Original Score: {original_result.value} (too strict)")
    print(f"Aligned Score: {aligned_result.value} (more reasonable)")
    
    # Log the aligned assessment for comparison
    mlflow.log_assessment(trace_id=test_trace.info.request_id, assessment=aligned_result)
    
    print("\n🎯 KEY INSIGHT:")
    print("The SIMBA optimizer learned from human feedback that the judge was being")
    print("too strict about exact matches. It rewrote the instructions to accept")
    print("conceptually accurate responses, not just exact text matches.")
    
else:
    print(f"\n⚠️ Need at least 10 traces for alignment (have {len(alignment_ready_traces)})")
    print("Make sure both judge evaluation and human feedback steps completed.")

2025/09/10 15:22:22 INFO dspy.teleprompt.simba: Starting batch 1 of 8.
2025/09/10 15:22:22 INFO dspy.teleprompt.simba: Sampling program trajectories on 10 examples x 6 samples.


Aligning Accuracy Judge with human feedback using SIMBA optimizer


📋 Context:
- Helpfulness judge: Already gives reasonable scores (4-5 range)
- Accuracy judge: Gives all 1s - needs alignment to fix overly strict scoring

We'll only align the accuracy judge to save time (alignment takes ~10 minutes)

Refreshing traces to get latest assessments...
Found 10 traces ready for accuracy judge alignment

📊 ALIGNING ACCURACY JUDGE

BEFORE ALIGNMENT:
--------------------
Original Instructions:

    Compare the response to the expected answer.
    
    Response: {{ outputs }}
    Expected: {{ expectations }}
    
    Score 1-5 based on accuracy:
    - 5: Matches expected answer closely
    - 4: Mostly accurate
    - 3: Partially accurate
    - 2: Some inaccuracies  
    - 1: Inaccurate
    

⏳ Starting alignment (this will take ~10 minutes)...
  0%|          | 0/60 [00:00<?, ?it/s]

2025/09/10 15:22:23 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['outputs']. Missing: ['expectations'].
2025/09/10 15:22:23 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['outputs']. Missing: ['expectations'].
2025/09/10 15:22:23 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['outputs']. Missing: ['expectations'].
2025/09/10 15:22:23 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['outputs']. Missing: ['expectations'].
2025/09/10 15:22:23 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['outputs']. Missing: ['expectations'].
2025/09/10 15:22:23 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['outputs']. Missing: ['expectations'].
2025/09/10 15:22:23 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['outputs']. Missing: ['e

Processed 1 / 60 examples:   2%|▏         | 1/60 [00:02<02:37,  2.66s/it]

2025/09/10 15:22:25 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['outputs']. Missing: ['expectations'].


...

Processed 69 / 70 examples:  97%|█████████▋| 68/70 [00:00<00:00, 187.36it/s]

2025/09/10 15:27:28 WARNING dspy.predict.predict: Not all input fields were provided to module. Present: ['outputs']. Missing: ['expectations'].


Processed 70 / 70 examples: 100%|██████████| 70/70 [00:00<00:00, 301.54it/s]

2025/09/10 15:27:28 INFO dspy.teleprompt.simba: Final trainset scores: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Best: 0.0 (at index 0)







✅ Alignment complete!

AFTER ALIGNMENT:
--------------------
Aligned Instructions:
Compare the response to the expected answer.

Response: {{ outputs }}
Expected: {{ expectations }}

Score 1-5 based on accuracy:
- 5: Matches expected answer closely
- 4: Mostly accurate
- 3: Partially accurate
- 2: Some inaccuracies  
- 1: Inaccurate

📈 Testing Aligned Judge:
--------------------
Original Score: 1 (too strict)
Aligned Score: 1 (more reasonable)

🎯 KEY INSIGHT:
The SIMBA optimizer learned from human feedback that the judge was being
too strict about exact matches. It rewrote the instructions to accept
conceptually accurate responses, not just exact text matches.


In [21]:
#original instructions:
print(judge2.instructions)


    Compare the response to the expected answer.
    
    Response: {{ outputs }}
    Expected: {{ expectations }}
    
    Score 1-5 based on accuracy:
    - 5: Matches expected answer closely
    - 4: Mostly accurate
    - 3: Partially accurate
    - 2: Some inaccuracies  
    - 1: Inaccurate
    


In [22]:
# return judge from align() call instructions:
print(aligned_accuracy_judge.instructions)

Compare the response to the expected answer.

Response: {{ outputs }}
Expected: {{ expectations }}

Score 1-5 based on accuracy:
- 5: Matches expected answer closely
- 4: Mostly accurate
- 3: Partially accurate
- 2: Some inaccuracies  
- 1: Inaccurate


## Summary

This notebook demonstrates the complete MLflow Judges workflow:

### ✅ **What We Did:**
1. Generated 10 ML chat traces with questions and responses
2. Added human expectations (ground truth) for each trace
3. Created two judges:
   - **Helpfulness Judge**: Rates how helpful responses are (gave all 5s)
   - **Accuracy Judge**: Compares to expectations (gave all 1s - too strict!)
4. Added human feedback showing the accuracy judge was scoring too low
5. Used SIMBA optimizer to align the accuracy judge with human feedback

### 🎯 **Key Learning:**
- **Problem**: Accuracy judge was too strict, expecting exact text matches
- **Solution**: SIMBA learned from human feedback and rewrote the instructions
- **Result**: Aligned judge now accepts conceptually accurate responses

### 🔍 **View Results:**
Check the MLflow UI at http://localhost:5000 to see all traces, assessments, and the before/after alignment comparison!